In [1]:
import os
import cv2
import numpy as np
import pytesseract
import fitz  # PyMuPDF

In [2]:
class OCREngine:
    Tessaract = "tessaract"
    EasyOCR = "easyocr"
    PaddleOCR = "paddleocr"

class IM_Ops:
    def __init__(self) -> None:
        pass

    def image_reader(self, file_path):
        image = cv2.imread(file_path)
        if image is None:
            print(f"Error with given image")
            return None
        print("Image is successfully read")
        return image

    def image_grayscale(self, image):
        " Converts the image to grayscale"
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    def morpher(self, image, kernel_size=5):
        '''
         Applies a morphological operation to the image to remove noise and gaps by
        erosion and dilation of the image
        '''
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

    def image_thresholding(self, image):
        '''
        Applies Otsu's thresholding to the image to convert it into a binary image.
        '''
        _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        return binary_image

    def image_sharpen(self, image):
        """
        Applies a sharpening filter to the image to enhance its edges.
        """
        kernel = np.array([[0, -1, 0], [-1, 30, -1], [0, -1, 0]])
        return cv2.filter2D(image, -1, kernel)

class File_Ops:
    def __init__(self) -> None:
        pass

    def save_as_txt(self, text: str, saving_path: str):
        """
        This function saves a given text as a TXT file at a specified path, ensuring that both
        the directory and the file exist before writing.
        """
        # Ensure the directory exists
        directory = os.path.dirname(saving_path)
        if not os.path.exists(directory):
            os.makedirs(directory)  # Create the directory if it doesn't exist

        # Ensure the file exists (create it if it doesn't)
        if not os.path.exists(saving_path):
            with open(saving_path, "w", encoding="utf8") as file:  # Create an empty file
                pass  # Just to ensure the file is created

        # Write the text to the file
        try:
            with open(saving_path, "a", encoding="utf8") as file:  # Append mode
                file.write(text + "\n")
                print(f"Text successfully saved to {saving_path}")
        except Exception as e:
            print(f"Error {e} while writing to TXT file")

class KannadaOCR:
    def __init__(self):
        self.im_ops = IM_Ops()
        self.file_ops = File_Ops()
        self.ocr_engine = OCREngine.Tessaract
        self.im_crop_dir = "C:/Kannada_OCR/Outputs/Crop"
        self.ocr_text_out = "C:/Kannada_OCR/Outputs/OCR Text"

    def image_to_ocr_text(self, image_path):
        image = self.im_ops.image_reader(image_path)
        if image is None:
            return None

        custom_config = r"--oem 3 --psm 6"
        text = pytesseract.image_to_string(image, lang="kan", config=custom_config)
        return text

    def PDF_reader(self, pdf_file_path):
        """
        Converts each page of a PDF into an image and saves them in a directory.
        """
        doc = fitz.open(pdf_file_path)  # Open the PDF document using PyMuPDF (fitz)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)  # Load the page
            picture = page.get_pixmap(dpi=300)  # Convert the page to an image (300 dpi)
            pic_path = f"C:/Kannada_OCR/Outputs/pdf_images/{page_num + 1:02d}_page.jpg"  # Save path for each image
            picture.save(pic_path)  # Save the image
            print(f"Saved image: {pic_path}")
        doc.close()
        print(f"Extracted and saved {len(doc)} images from the document")

    def preprocessor(self, file_name):
        """
        The function performs preprocessing on the image for OCR tasks.
        """
        image = self.im_ops.image_reader(file_name)
        if image is None:
            return None

        gray_image = self.im_ops.image_grayscale(image)
        morphed_image = self.im_ops.morpher(gray_image)
        binary_image = self.im_ops.image_thresholding(morphed_image)
        final_image = self.im_ops.image_sharpen(binary_image)
        return final_image

class OCRDriver:
    def __init__(self, input_file):
        self.kannada_ocr = KannadaOCR()
        self.file_ops = File_Ops()
        self.input_file = input_file
        self.image_output_dir = "C:/Kannada_OCR/Outputs/pdf_images"

        # Ensure directory for PDF-to-image conversion exists
        if not os.path.exists(self.image_output_dir):
            os.makedirs(self.image_output_dir)

    def run_ocr(self):
        """Main function to decide whether the input is an image or a PDF and process accordingly."""
        if self.input_file.lower().endswith('.pdf'):
            print(f"Processing PDF: {self.input_file}")
            self.process_pdf()
        else:
            print(f"Processing Image: {self.input_file}")
            self.process_image()

    def process_image(self):
        """Process an individual image file, apply OCR, and save the extracted text."""
        # Step 1: Preprocess the image
        preprocessed_image = self.kannada_ocr.preprocessor(self.input_file)
        if preprocessed_image is None:
            print(f"Error processing image: {self.input_file}")
            return

        # Step 2: OCR the preprocessed image
        ocr_text = self.kannada_ocr.image_to_ocr_text(self.input_file)

        # Step 3: Save the OCR text to the OCR Text directory
        base_name = os.path.basename(self.input_file).replace(' ', '_')
        text_file_path = os.path.join(self.kannada_ocr.ocr_text_out, f"{base_name}.txt")

        self.file_ops.save_as_txt(ocr_text, text_file_path)

        print(ocr_text)

    def process_pdf(self):
        """Process a PDF, convert pages to images, and apply OCR to each page."""
        # Step 1: Extract images from the PDF and store in the output directory
        self.kannada_ocr.PDF_reader(self.input_file)

        # Step 2: Loop through extracted images, process each, and append OCR results to a file
        pdf_base_name = os.path.basename(self.input_file).replace(' ', '_').replace('.pdf', '')
        text_file_path = os.path.join(self.kannada_ocr.ocr_text_out, f"{pdf_base_name}_ocr.txt")

        for image_file in sorted(os.listdir(self.image_output_dir)):
            image_path = os.path.join(self.image_output_dir, image_file)
            if image_file.endswith('.jpg'):
                print(f"Processing extracted PDF image: {image_path}")

                # Step 3: Preprocess the image
                preprocessed_image = self.kannada_ocr.preprocessor(image_path)
                if preprocessed_image is None:
                    print(f"Error processing image: {image_path}")
                    continue

                # Step 4: OCR the preprocessed image
                ocr_text = self.kannada_ocr.image_to_ocr_text(image_path)

                # Step 5: Append the OCR text to the output file for the PDF
                self.file_ops.save_as_txt(ocr_text, text_file_path)

In [3]:
if __name__ == "__main__":
    # Input file (image or PDF)
    input_file = "C:\\Kannada_OCR\\filename_2.jpg"  # Or .jpg, .png, etc.
    
    # Instantiate the OCRDriver and run the OCR process
    ocr_driver = OCRDriver(input_file)
    ocr_driver.run_ocr()

Processing Image: C:\Kannada_OCR\filename_2.jpg
Image is successfully read
Image is successfully read
Text successfully saved to C:/Kannada_OCR/Outputs/OCR Text\filename_2.jpg.txt
ಸೋಣಿಯ ನ್‌ವೆಫಲ್ಲ:
ದೋ ಎಂಜಿನ್‌ ವೃಫಲ್ಯ:
ನಾಲರು ಮೀನುಗಾರರ ರಕಣ
ದವ | ೦)
ಗ೦ಯಿನ್ಟ ಬೆಂ 101% ಇಲ್ಲಿನ' ' ಬುರ್ವಿಕೆಂ ನಿವಾಸಿಗಳಾದ" ಮಣಿಕಂಠ
ಬಂದರಿನಿಂದ ಮೀನುಗಾರಿಕೆಗೆ ತೆರಳಿದ್ದ ಖಾರ್ವಿ, ನಾಗಪ್ಪ ಖಾರ್ವಿ, ಪಾಂಡುರಂಗ
ದೋಣಿಯೊಂದು ಏಂಜಿನ್‌ ಕೆಟ್ಟು ಅರಬ್ಬಿ ಖಾರ್ವಿ ಹಾಗೂ ಬೇಲಿಕೇರಿ ನಿವಾಸಿ
ಸಮುದ್ರದಲ್ಲಿ ಸಿಕ್ಕಿ ಹಾಕಿಕೊಂಡ ಘಟನೆ ಸಂತೋಷ್‌ ಖಾರ್ವಿ ಅವರನ್ನು ದಡಕ್ಕೆ
ರವಿವಾರ ಬೆಳಗ್ಗೆ ಸಂಭವಿಸಿದ್ದು ಬಳಿಕ ಕರೆ ತಂದಿದ್ದಾರೆ. ಈ ದೋಣಿ ದಿನೇಶ್‌
ಅದರಲ್ಲಿದ್ದ ನಾಲ್ವರು ಮೀನುಗಾರರನ್ನು ಖಾರ್ವಿಯವರಿಗೆ ಸೇರಿದ್ದಾಗಿದೆ.
ಕರಾವಳಿ ಕಾವಲು ಪಡೆಯ ಪೊಲೀಸರು ತಪ್ಪಿದ ದುರಂತೆ
ಸುರಕ್ಷಿತವಾಗಿ ರಕ್ಷಣೆ ಮಾಡಿದ್ದಾರೆ. ಈ ದೋಣಿ ಎರಡು ಬೃಹತ್‌ ಬಂಡೆ
ರವಿವಾರ ಬೆಳಗ್ಗೆ ಮೀನುಗಾರಿಕೆಗೆ ಈ ಕಲ್ಲುಗಳ. ಮಧ್ಯದ ಸಮುದ್ರದಲ್ಲಿ
ದೋಣಿತೆರಳಿದ್ದು ಆದರೆ ಗಂಗೊಳ್ಳಿಯಿಂದ ಸಿಕ್ಕಿಹಾಕಿಕೊಂಡಿದ್ದು ಒಂದು ವೇಳೆ
ಸ್ವಲ್ಪ ದೂರದ ಅರಬ್ಬಿ ಸಮುದ್ರದಲ್ಲಿ ಯಾವುದಾದರೂ ಬಂಡೆ ಕಲ್ಲಿಗೆ ಢಿಕ್ಕಿ
ಎಂಜಿನ್‌ ಕೆಟ್ಟು ಹೋಗಿ, ಎರಡು ಬೃಹತ್‌ ಹೊಡೆಯುತ್ತಿದ್ದರೆ ಮತ್ತಷ್ಟು ಅನಾಹುತ
ಬಂಡೆ ಕಲ್ಲುಗಳ ಮಧ್ಯೆ ಸಿಕ್ಕಿಹಾಕಿಕೊಂಡಿತ್ತು. ಸಂಭವಿಸುವ ಸಾ

In [4]:
if __name__ == "__main__":
    # Input file (image or PDF)
    input_file = "C:\\Kannada_OCR\\01_page.jpg"  # Or .jpg, .png, etc.
    
    # Instantiate the OCRDriver and run the OCR process
    ocr_driver = OCRDriver(input_file)
    ocr_driver.run_ocr()

Processing Image: C:\Kannada_OCR\01_page.jpg
Image is successfully read
Image is successfully read
Text successfully saved to C:/Kannada_OCR/Outputs/OCR Text\01_page.jpg.txt
ಓಓ .!
8 ಫಿ ಆ
ವ್ರ ಟ್ರಿ ಲ್‌ ವಾ ಸಾ
ಔ ಣ ವ [ಸಿದಾ ಇ
[ಿ
|'`ಿರಿ ಹು ತಾಂ , ಏಕಿ ದಗ ಸಂ 'ಬಿತಿನಸ ತೆ ಒಳ್ಳೆ
ಗಾ ವು ಗಿಗ್ಮಿ
ಬ ಶಿ ೧ ೧ ಟ್ಛ ೌ ಘಂ |
:('`ಔ ರ್ಯ ಪ್ರ್ಞರಿ ಇ. ೪ ಮಿಕರುೂ, ಉಷ್ಯಾಜು ಅತ ರತ ಗಳಾದರು . ವಳ?
ಗ ನ
6
ಸ ಪ್ಪಫ್ಪ್ಮ್ಮ್‌ಾ* * ಾ  ಾೈಾೈಾೌ
ಎಸ ನ ಥ್ರ ವ
ವಃ ದ 4 ಬರಲ್ಲ ಮೂರ ವಾ್‌ ಯಗಳು ೪) ೦೨೬
|
ದ ಈ ಜ ಟ್ಟ ಕ ೧
ಪ್ರ ಜಾ] ರಗ ಗ ದಃ ಪಳ 5) ಆಲ್‌ ೨ದಿ|ಓೀ ದಿ
| ಇ ಕ್ಟ |
ರೂ ಗಗ್‌ ಉದಿಗ್ಣು. . ಉಮ್ರಾ ಬಿಧು, ಈಜಲು, ವರು 13 ಷ್ಟ ನ ಸುನ,
4 ಂ (
ರು ನಾ ಬಾರ್‌ ಅರ ದೇಶಂ ದ) ಯಾಹ್ಯಾ ಯೇ್ಲಂಭಬಟರ
ಮಾ ೬) ಛಿ ಮ
೧ ನ್‌ ಛೆ ಕ
೧ ಗ್ಕಿ “ಆ
ು ತ್‌
ಓಲ ಬೀ ( £ ಗೆ 1! ಬಎಟ್ಟಾ ಣ್ಬ್ಳ ಕ ನೀ ಜಗಕೆ.) ಯು ಬಾ ೬
ತ 3 ೧.
ಣಿ ೦
ಗಿ 6 ಬಪಿ9ಿ (0 ವ0ಎ.ಂ) ಓಟು ಆಂದೆ * ಕ್‌ ಅಒರಲ್ಪ
; ಸಿವಿಸಿ) 01.3 ಟಿಟಿ ಬಿ) 6 ತಾ ತೆ ಕ
೧
| ಚ್‌ ಜು: ಎ0ಬ ಆಮು ಜಸು ನ್‌ | ( ಕ |
ಕ! ಕ್ಷೆ ಕ "ಔ ಒನ್‌ 3 ಕ್ಷೆ
ಕ ಹ್‌ ತೆ 8 ಹ ನ ತೆ “ ( ಬ ಜಿ ನ . |
ಲ ಕ್ತ 2 ಕ ೆ ೫ || ಗ "81 ( ಕ
|
ಆ ೧/7 ಣೆ
ಓ ಜಮಾನ ಎಷು 22222222222222222222222222222222222252222222222222222222222222222222222ಐ2ಐಐಐಐಐಸ


In [5]:
if __name__ == "__main__":
    # Input file (image or PDF)
    input_file = "C:\\Kannada_OCR\\Poem.jpg"  # Or .jpg, .png, etc.
    
    # Instantiate the OCRDriver and run the OCR process
    ocr_driver = OCRDriver(input_file)
    ocr_driver.run_ocr()

Processing Image: C:\Kannada_OCR\Poem.jpg
Image is successfully read
Image is successfully read
Text successfully saved to C:/Kannada_OCR/Outputs/OCR Text\Poem.jpg.txt
ಕ್ಷಮಾಧಾತ್ರಿಯು ಮುನಿದರೆ.....
ಸಿಗಿದಾಯಿತು ಒಡಲು ನೆಲ-ಜಲ ಬಗೆದು,
ಬರಿದಾಯಿತು ಪ್ರಕೃತಿಯ ಹಸಿರುಸೀರೆಯ ಹೊಳಪು,
ಕುಡಿದಾಯಿತು ನೀರು ಅಂತರ್ಜಲವ ಕೊರೆದು-
ಕ್ಷಮಾಧಾತ್ರಿಯಾದೆ ನೀನು ಇವೆಲ್ಲವ ಮರೆತು.
ವಿಷಗಾಳಿಯು ಹರಡಲು ಕಾರ್ಯಾನೆಯ ಬಿಲದಿ,
ಕಸರಾಶಿಯ ಸುರಿದರು ಜಲಮೂಲದ ತಟದಿ,
ವೃಕ್ಷಕುಲವು ನಶಿಸಲು ಮಾನವನ ಹಠದಿ-
ಕ್ಷಮಾಧಾತ್ರಿಯಾದೆ ನೀನು ಇವೆಲ್ಲವ ಮರೆತು.
ನೀರ ಮೂಲವನ್ನೇ ತಿರುಗಿಸಿದರೂ,
ಮಣ್ಣಿನ ಫಲವತ್ತತೆಯ ಕೆಡಿಸಿದರೂ,
ಪರಿಸರದ ಸಮತೋಲನೆಯ ಕಡಿದರೂ-
ಕ್ಷಮಾಧಾತ್ರಿಯಾದೆ ನೀನು ಇವೆಲ್ಲವ ಮರೆತು.
ಬೆಳಕಿನ ಹಿಂದೆ ನಿಶೆಯ ಛಾಯೆಯಿರಲು,
58೧715೧3 8



In [6]:
if __name__ == "__main__":
    # Input file (image or PDF)
    input_file = "C:\\Kannada_OCR\\Poem2.jpg"  # Or .jpg, .png, etc.
    
    # Instantiate the OCRDriver and run the OCR process
    ocr_driver = OCRDriver(input_file)
    ocr_driver.run_ocr()

Processing Image: C:\Kannada_OCR\Poem2.jpg
Image is successfully read
Image is successfully read
Text successfully saved to C:/Kannada_OCR/Outputs/OCR Text\Poem2.jpg.txt
ಪ್ರಕೃತಿ

ದೇವಲೋಕ ಧರೆಗಿಳಿದು ನಿಂತ ಇದು ಸೊಗಸೊ,

ಕನಸೊ, ನೆನಸೊ ಇದು ಸತ್ಯಲೋಕದ ಬೆಳಕೊ,

ಏನು ಹಿತಮಿತ ಬೆಡಗು, ಪ್ರಕೃತಿಯ ರಮ್ಯ ಚೆಲುವು!

ಕಣ್ಮನ ಹೃದಯ ತುಂಬುವ ಒನಪು ವೈಯಾರ;

ಎಲ್ಲಿ ನೋಡಿದರಲ್ಲಿ ಪ್ರಕೃತಿ ಮೈತುಂಬಿ ನಿಂತು

ಉಬ್ಬುತಗ್ಗಿನ ಮಧ್ಯೆ, ಗಿಡಮರ ಸಂದಿಯ ಒಳಗೆ,

ನೀರಿನಲೆಗಳ ಮೇಲೆ, ಗುಡ್ಡ ಮೋಡ ಕಣಿವೆಯ ನಡುವೆ 1

ನೀಲ ಹಸುರನು ಹಬ್ಬಿ, ಮಂದ ಮಾರುತ ತೀಡಿ

ಕುಣಿದು ಹಾಡುವ ಮೋಡಿ, ಮನ ತಟ್ಟುತ್ತಿದೆ ನೋಡಿ.

ಭೂಮಿಯಾಕಾಶದ ನಡುವೆ ಬೆಳಕಿನೇಣಿಯ ಹಬ್ಬ,

ದೂರ ದಿಗಂತದ ತುಂಬ ಶುಭ್ರ ಮೋಡದ ನೃತ್ಯ

ಯಾವ ಕಲ್ಪನೆ ಜಾಡು ಹಿಡಿದು ಕೊಡುತ್ತಿದೆ ರೂಪ!

ಕಣ್ಣಿಗೆಟುಕದ ದೂರ ಹಕ್ಕಿ ಚಿಲಿಪಿಲಿ ಹಾಡು,

ಮನಸು ತುಂಬುವಾಹ್ಲಾದ, ಮಣ್ಣು ವಾಸನೆ ಸ್ವಾದ,

ಇದು ಪ್ರಕೃತಿಯೊ, ಪ್ರಕೃತಿಯ ಸ್ವರೂಪ ದರ್ಶನವೊ,

ಕಣ್ಣು ಬಿಟ್ಟಲ್ಲೆಲ್ಲ ಏನಿದು ಮೋಹಕ ಮಾಟ!

ನಾನು ಪ್ರಕೃತಿ ಬೆಸೆದು, ಸೌಂದರ್ಯದನುಭವವಾಗಿ

ಆನಂದ ಹೊಳೆಯಲಿ ಮುಳುಗಿ ಕರಗಿದುದರ ನೆನಪು. 1

ನಾನೀ ಚೆಲುವಲಿ ಬೆಸೆದು, ನಾನೆ ಪ್ರಕೃತಿಯಾಗಿ

ಹೃದಯ ಹಂದರ ಹಾಡಾಗಿ, ವಿಶ್ವದರ್ಶನ ಕಂಡೆ,

ನಾನೆಂಬ ಚಿಪ್ಪನು ಒಡೆದು, ದಿಶೆದಿಶೆಗಳ ತಬ್ಬಿ ಹಿಡಿದು

ತದ್ಯಾತ್ಮ ಭಾವದಿ ನಾನು ನಿಸರ